In [2]:
import funkcije
from bs4 import BeautifulSoup

# Pridobitev csv datoteke za zadnjih 100 oglasov na portalu avto.net (dne 26. 10. 2023)

Na portalu [avto.net](https://www.avto.net/) je posebna [stran](https://www.avto.net/Ads/results_100.asp?oglasrubrika=1&prodajalec=2), ki vsebuje zadnjih 100 objavljenih oglasov (stran se sproti spreminja). V tem jupyter zvezku pridobim html kodo iz te podstrani, iz nje pridobim podatke, ki me zanimajo (ime avta, 1.registracija, število prevoženih kilometrov, gorivo, menjalnik, moč motorja in cena), te podatke shranim v slovar in na koncu naredim csv datoteko, ki jo kasneje uporabim za analizo.



# 1. pridobivanje html kode
Najprej pridobim html kodo iz spletne podstrani, in jo shranim v datoteko 'zadnjih_100.html'

In [3]:
import requests

#brez headers, sem dobival 403 error
headers = {
    'Accept': 'text/html, image/avif, image/webp, image/apng, image/svg+xml, */ *; q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US, en;q=0.9',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Site': 'none',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
} 

data_request = requests.get('https://www.avto.net/Ads/results_100.asp?', headers=headers)
soup = BeautifulSoup(data_request.content, 'html.parser')
vsebina_html = soup.prettify()
with open("zadnjih_100.html", "w", encoding='utf-8') as f:
     print(vsebina_html, file=f)

# 2. Izluščanje podatkov iz html hode
Iz html kode za vsak avto izluščimo naslednje podatke: ime, leto prve registracije, število prevoženih km, gorivo, tip menjalnika, moč motorja in cena. Ceno in ime avta dodamo v svoj seznam, medtem ko druge podatke shranimo vse skupaj v en seznam (zaradi tipa zapisa podatkov v html kodi).

In [8]:
with open('zadnjih_100.html', 'r', encoding='utf-8') as f:
    vsebina = f.read()
    soup = BeautifulSoup(vsebina, 'html.parser')

#Pridelamo seznam z imeni vseh avtov (kot so ta zapisana na avto.net)
imena_avtov = []

#Ker imajo vsa imena avtov pred njimi značko <div class="GO-Results-Naziv ... to izkoristimo, da izluščimo imena ven
for element in soup.find_all(class_='GO-Results-Naziv'):
    imena_avtov.append(element.text.strip())

#Sedaj pa pridelamo še en seznam, z vsemi drugimi podatki (hočem, da mi pri motorju pobere samo konjske moči)
drugi_podatki = []
for element in soup.find_all('td', class_='pl-3'):
    drugi_podatki.append(element.text.strip())
#drugi_podatki je seznam tipa ['1.registracija', '2011', 'Prevoženih', '270000 km', 'Gorivo', 'bencinski motor']...

#hočem, da mi pri moči motorja pobere samo konjske moči,
drugi_podatki = funkcije.poberi_km(drugi_podatki)

#morebiten podatek samo v kW mi pretvori v KM (1 kW = 1.341 KM) in zaokroži na najbližje celo število
drugi_podatki = funkcije.pretvorjeno_v_km(drugi_podatki)

#Sedaj pa še cene avtov
cene_avtov = []

for element in soup.find_all(class_=['GO-Results-Price', 'GO-Results-Top-Price']):
    cene_avtov.append(element.text.strip())

#V kodi se cena za posamezen avto 2x ponovi (razen pri top ponudbi), zato uporabim funkcijo uredi_seznam, ki mi za vsak avto pusti samo eno ceno
cene_avtov = funkcije.uredi_seznam(cene_avtov)


#sedaj moramo v seznamu cene_avtov rešiti tip 'AKCIJSKA CENA\n          \n\n\n\n\n\n            13.990 €\n           \n\n            13.690 €' in 
#                                              '3.390 €\n           \n\n\n\n\n\n\n\n\n\n            oz. 51,00 EUR / mesec (*)'
#                                               '35.999 €\n           \n\n            oz. 29.507 € + DDV(*)'

cene_avtov = funkcije.poenostavi(cene_avtov)
cene_avtov = funkcije.odstrani_cene(cene_avtov)
drugi_podatki = funkcije.odstrani_enote(drugi_podatki)

AttributeError: module 'funkcije' has no attribute 'odstrani_cene'

# 3. Izdelava slovarja
Izdelamo slovar tipa {'ime avta':{'1. registracija': 2011, 'Prevoženih': '270000 km', 'Gorivo': 'bencinski motor', 'Menjalnik': 'avtomatski menjalnik', 'Moč motorja': '160 KM'}}. Slovar sem izbral zato, ker so se določeni podatki v kodi dvakrat ponovili. Nekateri avti pa tudi nimajo podatka o npr. prevoženih kilometrih. Če je bil prisoten kakšen drugi podatek (npr. pri električnih avtih 'Baterija': '52 kW'), ga v slovar nisem dodal.

In [ ]:
#naredimo seznam indeksov, na katerih se nahaja '1.registracija' v seznamu drugi_podatki
sez_indeksov = funkcije.na_katerih_indeksih_se_nahaja(drugi_podatki, '1.registracija')

slovar = {}
vmesni_slovar = {}   #za vsako ime avta zgeneriramo svoj vmesni slovar
seznam_kljucev = ['1.registracija', 'Prevoženih', 'Gorivo', 'Menjalnik', 'Motor']
i = 1
j = 0
for el in drugi_podatki:
    if i in sez_indeksov: #natanko tedaj, ko naletimo na prvo registracijo, hočemo začet nov vmesni slovar (pri vsakem avtu je ta člen)
        slovar[imena_avtov[j]] = vmesni_slovar
        vmesni_slovar = {}
        j += 1
    if drugi_podatki[i - 1] in seznam_kljucev:
        vmesni_slovar[drugi_podatki[i - 1]] = drugi_podatki[i]
    i += 1

#dodamo avtomatski menjalnik k avtom z električnim pogonom (ker na spletni strani tega podatka ni, vendar pa je za obdelavo to uporabno zapisati)
for vmesni_slovar in slovar.values():
    if 'elektro pogon' in vmesni_slovar.values():
        vmesni_slovar['Menjalnik'] = 'avtomatski menjalnik'

#dodamo cene na konec vsakega podslovarja
for i, (avto, podatki) in enumerate(slovar.items()):
    podatki['Cena'] = cene_avtov[i]
print(slovar)

# 4. Izdelava csv datoteke
Na koncu pridelam še csv datoteko, ki jo potem uporabim za obdelavo podatkov.

In [4]:
#Tu bomo iz slovarja napisali csv datoteko, ki jo bomo kasneje uporabili za analizo podatkov

import csv
with open('zadnjih_100.csv', 'w', newline='', encoding='utf-8') as g:
    zapis = csv.writer(g)
    
    #Napišemo prvo vrstico
    zapis.writerow(['Ime', '1.registracija', 'Prevoženih km', 'Gorivo', 'Menjalnik', 'Motor v KM', 'Cena v €'])
    
    #Zapišemo podatkev vse naslednje vrstice
    for ime, podatki in slovar.items():
        zapis.writerow([ime] + [podatki.get(kljuc, '') for kljuc in ['1.registracija', 'Prevoženih', 'Gorivo', 'Menjalnik', 'Motor', 'Cena']])

